# Imports and setting up viz

NB : conda env1 on PC, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_mapping import *

In [ ]:
rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '10m',edgecolor=(0, 0, 0, 0.3), facecolor='none')

# Load files

In [ ]:
date_min = '2010-01-01'
date_max = '2014-12-31'

## Format functions

In [ ]:
def format_LMDZ_MO(filename, name, color=None, date_min=None, date_max=None):
    """
    Format the LMDZ-MO output for plotting.
    """
    ds = xr.open_mfdataset(filename)
    ds.attrs['name'] = name
    ds.attrs['plot_color'] = color if color else 'blue'

    rename_dict = {'time_counter':'time'}
    ds = ds.rename(rename_dict)
    #restrict
    # ds = ds.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))
    if date_min is not None and date_max is not None:
        ds = ds.sel(time=slice(date_min, date_max))

    #edit and convert units
    ds['evap'] = ds['evap'] *3600 * 24
    ds['evap'].attrs['units'] = 'mm/d'
    ds['evap'].attrs['name'] = 'ET'

    ds['precip'] = ds['precip'] *3600 * 24
    ds['precip'].attrs['units'] = 'mm/d'
    ds['precip'].attrs['name'] = 'Precipitation'

    ds['t2m'] = ds['t2m'] - 273.15
    ds['t2m'].attrs['units'] = '°C'
    ds['t2m'].attrs['name'] = '2-m temperature'

    ds['fluxsens']= -ds['sens']
    ds['fluxsens'].attrs['units'] = 'W/m²'
    ds['fluxsens'].attrs['name'] = 'Sensible heat flux'

    ds['netrad'] = ds['LWdnSFC'] - ds['LWupSFC'] + ds['SWdnSFC'] - ds['SWupSFC']
    ds['netrad'].attrs['units'] = 'W/m2'

    ds['SWnetSFC'] = ds['SWdnSFC'] - ds['SWupSFC']
    ds['SWnetSFC'].attrs['units'] = 'W/m2'

    ds['LWnetSFC'] = ds['LWdnSFC'] - ds['LWupSFC']
    ds['LWnetSFC'].attrs['units'] = 'W/m2'

    ds['P - E'] = ds['precip'] - ds['evap']
    ds['P - E'].attrs['units'] = 'mm/d'

    return ds

## Sims 3 domain sizes (2010-2014)

In [ ]:
LAM_1000_40_dir='../../../JZ_simu_outputs/LAM/LAM_1000_40/noirr_2010_2022'
# LAM_1500_40_dir='../../../JZ_simu_outputs/LAM/LAM_1500_40_sim224'
LAM_1500_60_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/LAM_1500_60_sim206' #not the exact version used afterwards but appropriate for sensitivity comparison on domain size
LAM_2000_80_dir='../../../JZ_simu_outputs/LAM/LAM_2000_80/LAM_2000_80_sim199'

In [ ]:
#smaller LAM (original)
filename = '{}/ATM/TS*.nc'.format(LAM_1000_40_dir)
# filename = '{}/ATM/MO/*.nc'.format(LAM_1000_40_dir)

color = 'blue'

lmdz_1000_40 = format_LMDZ_MO(filename, 'LAM_1000km_NBP40', color=color, date_min=date_min, date_max=date_max)

lmdz_1000_40


In [ ]:
filename = '{}/ATM/MO/*.nc'.format(LAM_1500_60_dir)

lmdz_1500_60 = format_LMDZ_MO(filename, 'LAM_1500km_NBP60', color='green')

lmdz_1500_60

In [ ]:
filename = '{}/ATM/MO/*.nc'.format(LAM_2000_80_dir)

lmdz_2000_80 = format_LMDZ_MO(filename, 'LAM_2000km_NBP80', color='red')

lmdz_2000_80

## Forcing with LMDZ vs forcing with ERA5

LAM 1000km NBP40


2010-2012 ?

## Obs

### ERA5

In [ ]:
#open ERA5 file
filename = '../../../obs/ERA5/ERA5_large_eval.nc'
era = xr.open_mfdataset(filename)
era.attrs['name'] = 'ERA5'

#define or edit vars
#name ERA5 vars like in LMDZOR outputs
dict =  {
        'longitude':'lon',
        'latitude':'lat',
        'e':'evap',
        'ro':'totrunoff',
        'tp':'precip',
        'u10':'u10m',
        'v10':'v10m',
        'p71.162' : 'uq',
        'p72.162' : 'vq',
        'sshf' : 'fluxsens',
        'slhf' : 'fluxlat',
        'si10' : 'sfcWind',
        'ssrd' : 'SWdnSFC',
        'strd' : 'LWdnSFC',
        'ssr' : 'SWnetSFC',
        'str' : 'LWnetSFC',
        }
era = era.rename(dict)
era=era.sel(time=slice(date_min, date_max))
# era = era.sel(lon=slice(lon_min,lon_max),lat=slice(lat_max, lat_min))

#make evap positive in era
era['evap'] = -era['evap'] * 1000
era['evap'].attrs['units'] = 'mm/d'
era['precip'] = era['precip']*1000
era['precip'].attrs['units'] = 'mm/d'
# era['cldt'] = era['tcc']*100
# era['cldt'].attrs['units'] = '%'

era['P - E'] = era['precip'] - era['evap']
era['P - E'].attrs['units'] = 'mm/d'

era['calcWind'] = np.sqrt(era['u10m']**2 + era['v10m']**2)
era['calcWind'].attrs['units'] = 'm/s'

era['t2m'] = era['t2m'] - 273.15
era['t2m'].attrs['units'] = '°C'
era['t2m'].attrs['name'] = '2-m temperature'
   
#convert 4 radiation variables from J/m2 to W/m2
era['SWdnSFC'] = era['SWdnSFC'] / (3600 *24)
era['SWdnSFC'].attrs['units'] = 'W/m2'
era['LWdnSFC'] = era['LWdnSFC'] / (3600 *24)
era['LWdnSFC'].attrs['units'] = 'W/m2'
era['SWnetSFC'] = era['SWnetSFC'] / (3600 *24)
era['SWnetSFC'].attrs['units'] = 'W/m2'
era['LWnetSFC'] = era['LWnetSFC'] / (3600 *24)
era['LWnetSFC'].attrs['units'] = 'W/m2'
#add up vars and netrad
era['LWupSFC'] = - era['LWnetSFC'] + era['LWdnSFC']
era['LWupSFC'].attrs['units'] = 'W/m2'
era['SWupSFC'] = - era['SWnetSFC'] + era['SWdnSFC']
era['SWupSFC'].attrs['units'] = 'W/m2'
era['netrad'] = era['LWnetSFC'] + era['SWnetSFC']
era['netrad'].attrs['units'] = 'W/m2'

era


In [ ]:
#open tqruv file
filename='../../../obs/ERA5/TQRUV_850_2010_2022_monthly.nc'
tqruv = xr.open_dataset(filename)
tqruv.attrs['name'] = 'ERA5'
# tqruv = tqruv.sel(longitude=slice(lon_min,lon_max),latitude=slice(lat_max,lat_min))
tqruv = tqruv.sel(time=slice(date_min, date_max))
#rename longitude and latitude to lon and lat
tqruv = tqruv.rename({'longitude':'lon','latitude':'lat'})

#rename all variables to add 850 in name
tqruv['u850'] = tqruv['u']
tqruv['v850'] = tqruv['v']
tqruv['q850'] = tqruv['q']
tqruv['t850'] = tqruv['t']
tqruv['r850'] = tqruv['r']
tqruv

### GLEAM

In [ ]:
#gleam
# filename='../../../obs/GLEAMv4.1a/E_1980-2022_GLEAM_v3.8a_MO.nc'
filename='../../../obs/GLEAMv4.1a/E/*_MO.nc'
gleam=xr.open_mfdataset(filename)
gleam.attrs['name']='GLEAM'
# gleam = gleam0.sel(lon=slice(lon_min,lon_max),lat=slice(lat_max,lat_min))
#adapt period
gleam=gleam.sel(time=slice(date_min, date_max))
#name gleam vars like in LMDZOR outputs
dict =  {
        'E':'evap'
        }
gleam = gleam.rename(dict)
#convert mm/month to mm/day
gleam['evap']=gleam['evap']/30
gleam['evap'].attrs['units']='mm/d'
gleam

### FluxCom

In [ ]:
#FluxCom
filename='../../..//obs/FluxCom/ensemble_rs-_meteo_ALL/monthly/*.RS_METEO.EBC-*.MLM-ALL.METEO-ALL.720_360.monthly.*.nc'
fluxcom0=xr.open_mfdataset(filename)
fluxcom=fluxcom0.sel(lon=slice(lon_min,lon_max+0.25),lat=slice(lat_max,lat_min))
fluxcom=fluxcom.sel(time=slice(date_min, date_max))

fluxcom.attrs["name"]='FluxCom'

fluxcom['evap'] = fluxcom['LE'] * 0.408 # To convert from MJ/m2/d to mm/d
fluxcom['evap'].attrs['units'] = 'mm/d'

fluxcom['fluxlat'] = fluxcom['LE'] / 0.0864 # To convert from MJ/m2/d to W/m2
fluxcom['fluxlat'].attrs['units'] = 'W/m²'

fluxcom['fluxsens'] = fluxcom['H'] / 0.0864 # To convert from MJ/m2/d to W/m2
fluxcom['fluxsens'].attrs['units'] = 'W/m²'

fluxcom['netrad'] = fluxcom['Rn'] / 0.0864 # To convert from MJ/m2/d to W/m2
fluxcom['netrad'].attrs['units'] = 'W/m²'
fluxcom

### GPCC

In [ ]:
#gpcc
filename='../../../obs/precips/precip.mon.total.0.25x0.25.v2020.nc'
gpcc0=xr.open_mfdataset(filename)
gpcc0.attrs['name'] = 'GPCC'
gpcc=gpcc0.sel(time=slice(date_min, date_max))
gpcc['lon'] = ((gpcc['lon'] + 180) % 360) - 180
gpcc1 = gpcc.sortby('lon')

# gpcc=gpcc.sel(lon=slice(lon_min,lon_max+0.25),lat=slice(lat_min,lat_max))
gpcc=gpcc1.sel(lon=slice(lon_min,lon_max+0.25),lat=slice(lat_max,lat_min))

# gpcc1=gpcc.sel(lon=slice(347,360),lat=slice(lat_max,lat_min))
# gpcc2=gpcc.sel(lon=slice(0,lon_max+0.25),lat=slice(lat_max,lat_min))
# gpcc=xr.combine_by_coords([gpcc1, gpcc2])
# gpcc = gpcc.where(gpcc['lon'] < lon_max+0.25, drop=True)

# gpcc['lon']=gpcc['lon']-180.0
# gpcc=gpcc.sel(lon=slice(lon_min,lon_max+0.25))

gpcc['precip'] = gpcc['precip'] / 30 #convert to mm/d
gpcc['precip'].attrs['units'] = 'mm/d'
gpcc

# Masking, interpolation

In [ ]:
#define masks
con_mask=lmdz_2000_80['contfracATM']>0.95
ip_mask=polygon_to_mask(lmdz_2000_80, iberian_peninsula)

In [ ]:
lmdz_1000_40_i80=lmdz_1000_40.interp_like(lmdz_2000_80)
lmdz_1500_60_i80=lmdz_1500_60.interp_like(lmdz_2000_80)

In [ ]:
cont_lmdz_1000_40=lmdz_1000_40_i80.where(con_mask)
cont_lmdz_1500_60=lmdz_1500_60_i80.where(con_mask)
cont_lmdz_2000_80=lmdz_2000_80.where(con_mask)

ip_lmdz_1000_40 = lmdz_1000_40_i80.where(ip_mask, drop=False).where(con_mask)
ip_lmdz_1500_60 = lmdz_1500_60_i80.where(ip_mask, drop=False).where(con_mask)
ip_lmdz_2000_80 = lmdz_2000_80.where(ip_mask, drop=False).where(con_mask)


In [ ]:
lmdz_1000_40_era = lmdz_1000_40.interp_like(era)
# lmdz_1000_40_gpcc = lmdz_1000_40.interp_like(gpcc)
# lmdz_1000_40_gleam = lmdz_1000_40.interp_like(gleam)
# lmdz_1000_40_fluxcom = lmdz_1000_40.interp_like(fluxcom)
# lmdz_1000_40_tqruv = lmdz_1000_40.interp_like(tqruv)

lmdz_1500_60_era = lmdz_1500_60.interp_like(era)
# lmdz_1500_60_gpcc = lmdz_1500_60.interp_like(gpcc)
# lmdz_1500_60_gleam = lmdz_1500_60.interp_like(gleam)
# lmdz_1500_60_fluxcom = lmdz_1500_60.interp_like(fluxcom)
# lmdz_1500_60_tqruv = lmdz_1500_60.interp_like(tqruv)

lmdz_2000_80_era = lmdz_2000_80.interp_like(era)
# lmdz_2000_80_gpcc = lmdz_2000_80.interp_like(gpcc)
# lmdz_2000_80_gleam = lmdz_2000_80.interp_like(gleam)
# lmdz_2000_80_fluxcom = lmdz_2000_80.interp_like(fluxcom)
# lmdz_2000_80_tqruv = lmdz_2000_80.interp_like(tqruv)

In [ ]:
ip_interp=False
if ip_interp:
    ip_lmdz_1000_40_era = ip_lmdz_1000_40.interp_like(era)
    # ip_lmdz_1000_40_gpcc = ip_lmdz_1000_40.interp_like(gpcc)
    # ip_lmdz_1000_40_gleam = ip_lmdz_1000_40.interp_like(gleam)
    # ip_lmdz_1000_40_fluxcom = ip_lmdz_1000_40.interp_like(fluxcom)
    # ip_lmdz_1000_40_tqruv = ip_lmdz_1000_40.interp_like(tqruv)

    ip_lmdz_1500_60_era = ip_lmdz_1500_60.interp_like(era)
    # ip_lmdz_1500_60_gpcc = ip_lmdz_1500_60.interp_like(gpcc)
    # ip_lmdz_1500_60_gleam = ip_lmdz_1500_60.interp_like(gleam)
    # ip_lmdz_1500_60_fluxcom = ip_lmdz_1500_60.interp_like(fluxcom)
    # ip_lmdz_1500_60_tqruv = ip_lmdz_1500_60.interp_like(tqruv)

    ip_lmdz_2000_80_era = ip_lmdz_2000_80.interp_like(era)
    # ip_lmdz_2000_80_gpcc = ip_lmdz_2000_80.interp_like(gpcc)
    # ip_lmdz_2000_80_gleam = ip_lmdz_2000_80.interp_like(gleam)
    # ip_lmdz_2000_80_fluxcom = ip_lmdz_2000_80.interp_like(fluxcom)
    # ip_lmdz_2000_80_tqruv = ip_lmdz_2000_80.interp_like(tqruv)

In [ ]:
# era
era_i80=era.interp_like(lmdz_2000_80)
cont_era=era_i80.where(con_mask)
ip_era = cont_era.where(ip_mask, drop=False)

In [ ]:
#gleam
# gleam_i80=gleam.interp_like(lmdz_2000_80)
# cont_gleam=gleam_i80.where(con_mask)
# ip_gleam = cont_gleam.where(ip_mask, drop=False)


In [ ]:
#fluxcom
# fluxcom_i80=fluxcom.interp_like(lmdz_2000_80)
# cont_fluxcom=fluxcom_i80.where(con_mask)
# ip_fluxcom = cont_fluxcom.where(ip_mask, drop=False)
# orc_fluxcom=fluxcom.interp_like(orc)

In [ ]:
#gpcc
# int_gpcc=gpcc.interp_like(lmdz_2000_80)
# cont_gpcc=gpcc.interp_like(lmdz_1500_60).where(con_mask)
# ip_gpcc = cont_gpcc.where(ip_mask, drop=False)
# gpcc_iera = gpcc.interp_like(era)

In [ ]:
#era5 tqruv
# int_tqruv = tqruv.interp_like(lmdz_1500_60)
# cont_tqruv = tqruv.interp_like(lmdz_1500_60).where(con_mask)
# ip_tqruv = cont_tqruv.where(ip_mask, drop=False)

# Tests

## Maps

In [ ]:
#Display map of var for ds
var='evap'
ds=lmdz_1000_40
color_map=wet
max_value=None
min_value=None

map_ave(ds, var, cmap=color_map, vmin=min_value, vmax=max_value, hex=False)

In [ ]:
#Diff map
var='precip'
max_value= 2
min_value=-2
ds1=lmdz_2000_80_era
ds2=era
color_map=emb

title='Evapotranspiration bias (mm/d, {} - {})'.format( ds1.attrs['name'], ds2.attrs['name'])

map_diff_ave(ds1, ds2, var, vmin=min_value, vmax=max_value, title=title, cmap=color_map, hex=False, sig=False)

diff=ds1[var]-ds2[var]
title='{} bias, {} vs {} ({})'.format( var, ds1.attrs['name'], ds2.attrs['name'], ds1[var].attrs['units'])
map_seasons(diff, cmap=color_map,  vmin=min_value, vmax=max_value, title=title, hex=False)

In [ ]:
#map for 4 seasons
var='precip'
ds1=cont_sim
ds2=cont_era
max_value=2
min_value=None
# min_value=-max_value

diff=ds1[var]-ds2[var]
title='{} bias, {} vs {} ({})'.format( var, ds1.attrs['name'], ds2.attrs['name'], ds1[var].attrs['units'])
plotvar=diff

# plotvar=ds1[var]
# title='P-E (mm/d, {})'.format(ds1.attrs['name'])

map_seasons(plotvar, cmap=emb, vmax=max_value, title=title, hex=False)

In [ ]:
# Display 2 maps of var
var = 'precip'
min_value=None
max_value=6
color_map=wet
ds1=era
ds2=lmdz_1500_60
map_two_ds(ds1, ds2, var, vmin=min_value, figsize=(15,6), vmax=max_value, cmap=color_map, hex=True)

In [ ]:
#Display map of var for ds on restricted area
var='precip'
ds = lmdz_2000_80_era
latmin=42.2
latmax=43.5
lonmin=-2
lonmax=3
vmax= 3
vmin=-3

#restrict ds to latmin, latmax, lonmin, lonmax
ds = ds.where(ds.lat >= latmin, drop=True).where(ds.lat <= latmax, drop=True)
ds = ds.where(ds.lon >= lonmin, drop=True).where(ds.lon <= lonmax, drop=True)
plotvar = ds[var].mean(dim='time') -era[var].mean(dim='time')
map_plotvar(plotvar, cmap=emb, vmin=vmin, vmax=vmax, title=ds.attrs['name'])

In [ ]:
#Relative diff map
var='precip'
max_value=None
ds1=cont_fluxcom
ds2=ip_fluxcom
# plt.title('Gross primary production relative difference (%)')
map_rel_diff_ave(ds1, ds2, var, vmax=max_value)

In [ ]:
ds=sim
map_wind(ds, height='10m', scale=50)

In [ ]:
ds=sim_era
map_moisture_transport(ds, scale=1500)

In [ ]:
ds=geopt
plotvar = ds['z']
map_plotvar(plotvar, cmap='terrain', vmin=0.0, vmax=2400, title='Geopotential height (m)')

## Time series

In [ ]:
var='precip'

ds1=sim_1000_iera_pyrenees
ds2=sim_1500_iera_pyrenees
ds3=sim_2000_iera_pyrenees
ds4=era_pyrenees
ds5=gpcc_pyrenees

# ds4=ip_gpcc

ds6=ip_fluxcom
ds7=ip_gleam

# ds1=ip_orc
# ds2=ip_orcirr
# ds3=ip_fluxcom_orc

ds_list=[ds1, ds2, ds3, ds4, ds5]
# ds_list=[ds1, ds2, ds3, ds4, ds5, ds6, ds7]
year_max=2014
time_series_ave(ds_list, var, year_max=year_max, title='{} {}'.format(var, ds1[var].attrs['units']))
seasonal_cycle_ave(ds_list, var, year_max=year_max, title='Seasonal cycle of {} ({})'.format(var, ds1[var].attrs['units']))

# Figures

## 3 maps for 3 sizes, whole domain, diff with ERA5 : precip, evap, t2m

In [ ]:
savefig=True
#Diff map
vars=['precip', 'evap']
max_value=0
min_value=4
ds_list=[era, lmdz_1000_40_era, lmdz_1500_60_era, lmdz_2000_80_era]
color_map=bluesW

title='off'

for var in vars:
    label='{} ({})'.format(ds1[var].attrs['name'], ds1[var].attrs['units'])
    for ds1 in ds_list:
        map_ave(ds1, var, vmin=min_value, vmax=max_value, title=title, clabel=label, cmap=color_map)
        if savefig:
            plt.savefig('figures/chap4/var_map_{}_{}.png'.format(var, ds1.attrs['name']), bbox_inches='tight', dpi=300)

In [ ]:
savefig=True
#Diff map
vars=['precip', 'evap']
max_value= 1.5
min_value=-1.5
ds_list=[lmdz_1000_40_era, lmdz_1500_60_era, lmdz_2000_80_era]
ds2=era
color_map=emb_neutral

title='off'

for var in vars:
    label='{} difference ({})'.format(ds1[var].attrs['name'], ds1[var].attrs['units'])
    for ds1 in ds_list:
        map_diff_ave(ds1, ds2, var, vmin=min_value, vmax=max_value, title=title, clabel=label, cmap=color_map, sig=False)
        if savefig:
            plt.savefig('figures/chap4/diff_map_{}_era_{}.png'.format(var, ds1.attrs['name']), bbox_inches='tight', dpi=300)

In [ ]:
savefig=False
#Diff map
vars=['precip', 'evap']
max_value= 100
min_value=-100
ds_list=[lmdz_1000_40_era, lmdz_1500_60_era, lmdz_2000_80_era]
ds2=era
color_map=emb_neutral

title='off'

for var in vars:
    label='{} relative difference (%)'.format(ds1[var].attrs['name'])
    for ds1 in ds_list:
        map_rel_diff_ave(ds1, ds2, var, vmin=min_value, vmax=max_value, title=title, clabel=label, cmap=color_map)
        if savefig:
            plt.savefig('figures/chap4/rel_diff_map_{}_era_{}.png'.format(var, ds1.attrs['name']), bbox_inches='tight', dpi=300)

## Same for t2m

In [ ]:
savefig=True
#Diff map
vars=['t2m']
min_value=0
max_value=4
min_value, max_value = None, None

ds_list=[era, lmdz_1000_40_era, lmdz_1500_60_era, lmdz_2000_80_era]
color_map=reds

title='off'

for var in vars:
    label='{} ({})'.format(lmdz_1000_40_era[var].attrs['name'], lmdz_1000_40_era[var].attrs['units'])
    for ds1 in ds_list:
        map_ave(ds1, var, vmin=min_value, vmax=max_value, title=title, clabel=label, cmap=color_map)
        if savefig:
            plt.savefig('figures/chap4/var_map_{}_{}.png'.format(var, ds1.attrs['name']), bbox_inches='tight', dpi=300)

In [ ]:
savefig=True
#Diff map
vars=['t2m']
min_value=-2.0
max_value= 2.0
# min_value, max_value = None, None

ds_list=[lmdz_1000_40_era, lmdz_1500_60_era, lmdz_2000_80_era]
ds2=era
color_map=emb

title='off'

for var in vars:
    label='{} difference ({})'.format(ds1[var].attrs['name'], ds1[var].attrs['units'])
    for ds1 in ds_list:
        map_diff_ave(ds1, ds2, var, vmin=min_value, vmax=max_value, title=title, clabel=label, cmap=color_map, sig=False)
        if savefig:
            plt.savefig('figures/chap4/diff_map_{}_era_{}.png'.format(var, ds1.attrs['name']), bbox_inches='tight', dpi=300)